In [1]:
from myABBR import * ; import myUTIL as mu
# A. 市場レートとの差を戻す関数
def calcDIFF(yld):               
  # make crvOBJ and calc annuity
  calcDF = [np.exp(yy) for yy in yld]
  crvOBJ = ql.DiscountCurve([DTset]+DTtgt, [1.0]+calcDF,dcA365,calJP)
  DF1m, DF6m, DF7m, DF1y, DF15y, DF2y = \
                            [crvOBJ.discount(dt) for dt in DTcalcDF]
  ann2y  = mu.calcAnnuity(sw2SCD, crvOBJ)
  # calc difference of implied rate and market rate
  dTb6m = (1.0 /DF6m -1.0) /yr6m  - tb6mRT
  dFr17 = (DF1m/DF7m -1.0) /yr1x7 - fr17RT
  dSw2y = (1.0 - DF2y)     /ann2y - sw2yRT
  return nA([dTb6m, dFr17, dSw2y])

# B. 日付, 市場レート等の初期値設定
DTtrade = jDT(2025,12,19); DTset=calJP.advance(DTtrade,Tp2,DD)

tb6mRT,    fr17RT,  sw2yRT,  iRT,  eps,  tol, maxIter           = \
0.0107076, 0.01115, 0.0143, 0.001, 1e-7, 1e-14, 30 ; setEvDT(DTtrade)

# B-1. 日付(DTxx)
sw2SCD = ql.Schedule(DTset, calJP.advance(DTset,2,YY), 
                        pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)
fr1SCD = ql.Schedule(calJP.advance(DTset,1,MM), calJP.advance(DTset,7,MM), 
                        pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)
DT6m, DT1y, DT1y6m, DT2y = [sw2SCD[ii] for ii in range(1,len(sw2SCD))]
DT1m, DT7m               = [fr1SCD[ii] for ii in range(  len(fr1SCD))]
DTtgt    = [      DT6m, DT7m,               DT2y]   # ターゲットの日付
DTcalcDF = [DT1m, DT6m, DT7m, DT1y, DT1y6m, DT2y]   # 要DF計算の日付
# B-2. 年数(yrxx)
yr1m, yr6m, yr7m, yr1y, yr1y6m, yr2y = \
                   [dcA365.yearFraction(DTset,dt) for dt in DTcalcDF]
yr1x7 =  dcA365.yearFraction(DT1m,DT7m)

# C. ニュートン法    (ディスカウントファクタexp[-rt]の指数部分のレートが変数 )
yld = -iRT*nA([yr6m, yr7m, yr2y]); nYLD = len(yld)      # 初期値
# C-1. ループ
try:
  for ii in range(maxIter):
    fx   = calcDIFF(yld)
    diff = np.max(np.abs(fx)); print(f'#{ii} fx={nA(fx)} diff={diff:.2e}')
    if diff < tol: 
                  print('*** found yld *** \n'); break;  # この"yld"が解
    # FD Jacobian
    JC           = np.zeros((nYLD, nYLD))
    for jj in range(nYLD):
        yCP      = yld.copy() 
        yCP[jj] += eps
        fx1      = calcDIFF(yCP)   
        JC[:,jj] = (fx1-fx)/eps  
    dy = np.linalg.solve(JC, -fx)                         # ニュートンステップ
    yld = yld + dy                
except:
  raise RuntimeError('Newton did not converge.')

# D. 見つかった解(yld)でDFを算出
calcDF = [float(np.exp(yy)) for yy in yld]
crvOBJ = ql.DiscountCurve([DTset]+DTtgt, [1.0]+calcDF, dcA365, calJP)
nSetF(10) ; print(nA([crvOBJ.discount(dt) for dt in DTtgt])) ; nSetF()
print([dt.ISO() for dt in DTtgt])

#0 fx=[-0.00971 -0.01015 -0.0133 ] diff=1.33e-02
#1 fx=[0.00002 0.00003 0.00003] diff=3.17e-05
#2 fx=[0. 0. 0.] diff=1.86e-10
#3 fx=[ 0. -0.  0.] diff=2.34e-16
*** found yld *** 

[0.9946892229 0.9935996284 0.9718779323]
['2026-06-23', '2026-07-23', '2027-12-23']


* np.linalg.solveの例

In [2]:
A = nA([[2.0000456, 1.0],
        [1.0, 3.0]])
b = nA([1.0, 5.0])
x = np.linalg.solve(A, b) ; print('x=',x)
print('for checking: A @ x =', A @ x, '= b')

x= [-0.39999  1.8    ]
for checking: A @ x = [1. 5.] = b
